In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import math

base_path = "../build/03_sine_waves"

values = pd.read_csv(f"{base_path}/combined.csv")

values['time'] = values.index * (1/44100)



values[-5000:].plot(x='time', figsize=(30, 5))


In [ ]:
fft = pd.read_csv(f"{base_path}/mags.csv", names=['Frequency Bins'])
fft['freq'] = fft.index * (44100)/(1024 * 1024)
fft.plot(x='freq', y='Frequency Bins', figsize=(10,5))

In [ ]:
# Create a Hann window
x = np.linspace(0, len(values.index), len(values.index))
y = 0.5 * (1 - np.cos(2.0 * np.pi * x / len(values.index)))

plt.plot(x, y)

In [ ]:
fft = pd.read_csv(f"{base_path}/raw.csv", names=['Frequency Bins'])
fft['db'] = 20 * np.log10(fft['Frequency Bins'])

sample_rate = 48000
n_bins = 8192
step = 1024
frequency_cutoff = 2000
time_per_fft = step / sample_rate

frequencies = np.fft.fftfreq(n_bins, 1 / sample_rate)

hz_per_bin = sample_rate / n_bins
freq_bin_end = int(math.ceil(frequency_cutoff / hz_per_bin))

num_frames = len(fft) // freq_bin_end
fft_data = fft['db'].values.reshape(num_frames, freq_bin_end)

fig, ax = plt.subplots()
line, = ax.plot(frequencies[:freq_bin_end], np.zeros(freq_bin_end))
ax.set_xlim(0, frequency_cutoff)  # Limit x-axis to half of Nyquist frequency
#ax.set_ylim(0, 1)  # Adjust based on expected FFT magnitude
ax.set_ylim(0, fft_data.max())   # Adjust based on expected FFT magnitude
def update(frame):
    # Update the FFT data in the plot
    print(f'Frame: {frame}')
    line.set_ydata(fft_data[frame, :freq_bin_end])
    return line,

ani = FuncAnimation(fig, update, frames=range(num_frames), interval=time_per_fft * 1000, blit=True)
print(f"Saving at {(1/time_per_fft)} fps")
ani.save('animation.mp4', writer='ffmpeg', fps=(1/time_per_fft))